<a href="https://colab.research.google.com/github/muhammaddurri/Praktikum-Machine-Learning-/blob/main/UASML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download misrakahmed/vegetable-image-dataset

Dataset URL: https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset
License(s): CC-BY-SA-4.0
 83% 445M/534M [00:03<00:02, 42.7MB/s]
100% 534M/534M [00:03<00:00, 156MB/s] 


# Task
Compare the performance of ResNet and ViT models for vegetable freshness detection based on digital images.

## Menyiapkan

### Subtask:
Unduh dan ekstrak dataset gambar sayuran.


**Reasoning**:
Extract the downloaded zip file and list the contents to verify the extraction.



In [ ]:
import zipfile
import os

zip_path = '/content/vegetable-image-dataset.zip'
extract_path = '/content/vegetable-image-dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(os.listdir(extract_path))

['Vegetable Images']


**Reasoning**:
List the contents of the 'Vegetable Images' directory to further verify the dataset extraction.



In [ ]:
print(os.listdir(os.path.join(extract_path, 'Vegetable Images')))

['test', 'validation', 'train']


## Persiapan data

### Subtask:
Muat dan pra-proses data gambar untuk pelatihan model.


**Reasoning**:
Import necessary libraries and define the dataset paths.



In [ ]:
import tensorflow as tf
from tensorflow import keras

train_dir = '/content/vegetable-image-dataset/Vegetable Images/train'
validation_dir = '/content/vegetable-image-dataset/Vegetable Images/validation'
test_dir = '/content/vegetable-image-dataset/Vegetable Images/test'

**Reasoning**:
Load the image datasets using image_dataset_from_directory with specified parameters.



In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)

class_names = train_ds.class_names
print(class_names)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.
['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']


## Pembanguunan model resnet50

### Subtask:
Bangun model ResNet50 untuk klasifikasi gambar.


**Reasoning**:
Build the ResNet50 model for image classification according to the instructions.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import regularizers

# Load the base ResNet50 model without the top layer
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)

# Freeze the base model weights
base_model_resnet.trainable = False

# Create the sequential model
model_resnet = Sequential([
    # ResNet50 handles its own preprocessing internally, no explicit preprocessing layer needed here
    base_model_resnet,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model_resnet.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

model_resnet.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 15)             │         7,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,644,495 (94.01 MB)

 Trainable params: 1,056,783 (4.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

## Training model resnet

### Subtask:
Latih model ResNet50 menggunakan data yang telah disiapkan.


**Reasoning**:
Train the ResNet50 model using the prepared training and validation datasets for a specified number of epochs, store the training history, and display it.



In [ ]:
epochs = 15
history_resnet = model_resnet.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=epochs
)

import pandas as pd
history_df = pd.DataFrame(history_resnet.history)
display(history_df)

Epoch 1/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 144ms/step - accuracy: 0.8892 - loss: 1.0452 - val_accuracy: 0.9927 - val_loss: 0.3535
Epoch 2/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 49s 104ms/step - accuracy: 0.9821 - loss: 0.3435 - val_accuracy: 0.9907 - val_loss: 0.2511
Epoch 3/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 48s 103ms/step - accuracy: 0.9834 - loss: 0.2579 - val_accuracy: 0.9937 - val_loss: 0.2063
Epoch 4/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 49s 103ms/step - accuracy: 0.9821 - loss: 0.2436 - val_accuracy: 0.9947 - val_loss: 0.2003
Epoch 5/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 48s 103ms/step - accuracy: 0.9835 - loss: 0.2346 - val_accuracy: 0.9937 - val_loss: 0.2045
Epoch 6/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 48s 103ms/step - accuracy: 0.9848 - loss: 0.2208 - val_accuracy: 0.9960 - val_loss: 0.1910
Epoch 7/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 48s 103ms/step - accuracy: 0.9856 - loss: 0.2108 - val_accuracy: 0.9937 - val_loss: 0.1931
Epoch 8/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 49s 104ms/step - accuracy: 0.9816 - loss: 0

,accuracy,loss,val_accuracy,val_loss
0,0.953067,0.673152,0.992667,0.353472
1,0.981200,0.318725,0.990667,0.251066
2,0.982667,0.257846,0.993667,0.206336
3,0.982333,0.242562,0.994667,0.200346
4,0.981000,0.244332,0.993667,0.204496
5,0.983400,0.227556,0.996000,0.191000
6,0.982933,0.220886,0.993667,0.193129
7,0.981933,0.234845,0.994333,0.179092
8,0.984533,0.206447,0.991667,0.184195
9,0.983600,0.205208,0.990000,0.195229


**Reasoning**:
The task requires analyzing the data, so I need to load the dataset first.



In [ ]:
!pip install transformers datasets torch torchvision


In [ ]:
from transformers import ViTForImageClassification, ViTImageProcessor
from datasets import load_dataset
import torch
from torchvision import transforms
from torch.utils.data import DataLoader


In [ ]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("/content/drive/datasetsaya/Vegetable Images/train", transform=transforms.ToTensor())
test_dataset = ImageFolder("/content/drive/datasetsaya/Vegetable Images/test", transform=transforms.ToTensor())
validation_dataset = ImageFolder("/content/drive/datasetsaya/Vegetable Images/validation", transform=transforms.ToTensor())

In [ ]:
from transformers import ViTForImageClassification, ViTImageProcessor

# Memuat pre-trained model dan image processor
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(train_dataset.classes),  # jumlah kelas (misalnya 15 sayur)
    ignore_mismatched_sizes=True # Add this argument to ignore the size mismatch in the final layer
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([15, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset.transform = transform
test_dataset.transform = transform
validation_dataset.transform = transform

In [ ]:
from torch.utils.data import DataLoader

# Buat DataLoader untuk setiap subset data
train_loader = DataLoader(
    train_dataset,
    batch_size=16,     # jumlah data per batch (bisa ubah sesuai RAM)
    shuffle=True,      # acak data saat training
    num_workers=2,     # paralel loading (sesuaikan jika di Colab)
    pin_memory=True
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=16,
    shuffle=False,     # tidak perlu diacak saat validasi
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,     # tetap urutan asli saat testing
    num_workers=2,
    pin_memory=True
)



In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # ganti sesuai kebutuhan
    model.train()
    for images, labels in train_loader:
        inputs = processor(images, return_tensors="pt").to(device)
        labels = labels.to(device)
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1} selesai")


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Epoch 1 selesai
Epoch 2 selesai
Epoch 3 selesai
